In [1]:
# Function that does n iterations of RMS calculation

import numpy as np
import matplotlib.pyplot as plt

def mc_rms (y_array1, y_array2, unc1, unc2, n):
    plt.clf()
    
    RMS=[]

    for w in range(n):
        y_array1_rand = y_array1 + (np.random.randn(len(y_array1)) * unc1)
        y_array2_rand = y_array2 + (np.random.randn(len(y_array2)) * unc2)
        diff = y_array1_rand - y_array2_rand
        rms = (np.sum(diff**2))/(len(diff))**(0.5)
        RMS.append(rms)
       
    avg = np.mean(RMS)
    error = np.std(RMS)
#     print('RMS = ',avg, '+/-', error)
#     print(avg)
    return [avg, error, RMS]
    
    RMS.sort()
#     plt.hist(RMS, bins=50)
#     plt.ylabel('Frequency')
#     plt.xlabel('RMS')
#     plt.show()

#     label = '/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+'.png'

#     plt.savefig(label)
    
#     print(RMS)

In [38]:
# Derived from ycfq_auto:

#yfcq_auto
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy.interpolate import interp1d
# import BDdb
import astrodb
import pandas as pd
db = astrodb.Database('/Users/victoriaditomasso/Desktop/BDNYCdeprecated.db')

# The inputs are the target source id, the spec order being compared and the path to the text file with the comparison sample (has to be tab delimited)
def yfcq(tar_source_id, spec_order, path_to_comp_sample_dataframe):

# This clears the figure, useful when saving the plots
    plt.clf()

    data_tar = db.query("select sources.id, sources.shortname, spectra.wavelength, spectra.flux, spectra.unc, radial_velocities.radial_velocity from sources join spectra on sources.id=spectra.source_id join radial_velocities on spectra.source_id=radial_velocities.source_id where spectra.source_id={} and spectra.wavelength_order={}".format(tar_source_id, spec_order))

    df=pd.read_csv(path_to_comp_sample_dataframe,sep='\t')

    rms_100s=[]
    rms_200s=[]
    rms_500s=[]
    rms_1000s=[]

    for i, row in df.iterrows() :

        plt.clf()
        
#Gets the wavelength, flux, and uncertainty for the spec_ids listed in comp_sample
        data_comp = db.query("select spectra.wavelength, spectra.flux, spectra.unc from spectra where spectra.id={}".format(row['spec_id']))
    
# Separates out the wavelength array, flux array, RV and uncertainty as a float for the two spectra	
        w_tar = np.asarray(data_tar[0][2])
        f_tar = np.asarray(data_tar[0][3])

        w_comp = np.asarray(data_comp[0][0])
        f_comp = np.asarray(data_comp[0][1])

        rv_tar = data_tar[0][5]
        rv_comp = row['rv']

        unc_tar = data_tar[0][4]
        unc_comp = data_comp[0][2]

# Shifts the two spectra based on their RVs
        shifted_w_tar = (w_tar)*(1.-(rv_tar/2.99792458e5))
        shifted_w_comp = (w_comp)*(1.-(rv_comp/2.99792458e5))

# Interpolates the flux for the comp object so that when I subtract it from the target flux, I will get a properly calculated residual
# Remember: once you interpolate, you need to plot the w_tar vs the interpolated flux NOT w_comp vs interpolated flux
        x = shifted_w_tar
        xp = shifted_w_comp
        fp = f_comp
        f_comp_interp = np.interp(x, xp, fp)

#Checking if unc arrays are actually snrs and, if they are, converting them by doing 1/unc
        unc=np.asarray(unc_comp)
        avg_unc = np.sum(unc)/len(unc)
        if avg_unc > 4.0:
            unc_comp = 1.0/unc
#         if avg_unc == 1.0:
#             print 'UNC AVG = 1:'

# Interpolate the unc
        w = shifted_w_tar
        wp = shifted_w_comp
        up = unc_comp
        unc_comp_interp = np.interp(w, wp, up)

# Finds a normalization coefficient	
        dk = sum((f_tar * f_comp)/((unc_tar)**2 + (unc_comp_interp)**2))/sum((f_comp * f_comp)/((unc_tar)**2 + (unc_comp_interp)**2))

# Creates an array of normalized flux for the comparison object
        f_comp_norm_dk = f_comp * dk
        f_comp_norm_dk_interp = f_comp_interp * dk

# Sets the first and last flux point that will be used in quantification calculation/will be plotted
        f=50
        l=1000
        
# MC_rms calculation
        plot_title = str(row['shortname']) + str(data_tar[0][1])
        label = '/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+'.png' 
        
        y_array1 = f_tar[f:l]
        y_array2 = f_comp_norm_dk_interp[f:l]
        unc1 = unc_tar[f:l]
        unc2 = unc_comp_interp[f:l]
        
        
#         nrows = 4
#         fig, axes = plt.subplots(nrows,1)
    
#         for row,i in zip(axes,[100,200,500,1000]):
#             RMS=[]
#             rms = mc_rms(y_array1, y_array2, unc1, unc2, i)
#             RMS.append(rms[0])
            
#             plt.hist(rms[2],bins='auto')
#             plt.ylabel('Frequency')
#             plt.xlabel('RMS')
            
#             column_title = 'RMS n=' + str(i)
#             df[column_title] = rms[0]
#             df.to_csv(str(data_tar[0][1])+'_'+str(spec_order)+'testing_mc.txt',sep='\t')

#         plot_title=str(row['shortname']) + '_' + str(data_tar[0][1])
#         plt.savefig('/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+'.png')
        
#         for i,j in zip([100,200,500,1000],[411,412,413,414]):
#             rms = mc_rms(y_array1, y_array2, unc1, unc2, i)
            
#             plt.subplot(j)
#             plt.hist(rms[2],bins='auto')
#             plt.title(i)
            
#             column_title = 'RMS n=' + str(i)
#             df[column_title] = rms[0]
#             df.to_csv(str(data_tar[0][1])+'_'+str(spec_order)+'testing_mc.txt',sep='\t')
            
#         plot_title=str(row['shortname']) + '_' + str(data_tar[0][1])
#         plt.savefig('/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+'.png')
            
          
        n=100
        rms_100 = mc_rms (y_array1, y_array2, unc1, unc2, n)
        rms_100s.append(rms_100[0])
        plt.subplot(411)
        plt.hist(rms_100[2], bins=50)
        plt.ylabel('Frequency')
        plt.xlabel('RMS')
        plt.title(str(plot_title)+str(n))
#         plt.savefig('/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+str(n)+'.png')
        
        m=200
        rms_200 = mc_rms (y_array1, y_array2, unc1, unc2, m)
        rms_200s.append(rms_200[0])
        plt.subplot(412)
        plt.hist(rms_200[2], bins=50)
        plt.ylabel('Frequency')
        plt.xlabel('RMS')
        plt.title(str(plot_title)+str(m))
#         plt.savefig('/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+str(m)+'.png')
        
        q=500
        rms_500 = mc_rms (y_array1, y_array2, unc1, unc2, q)
        rms_500s.append(rms_500[0])    
        plt.subplot(413)
        plt.hist(rms_500[2], bins=50)
        plt.ylabel('Frequency')
        plt.xlabel('RMS')
        plt.title(str(plot_title)+str(q))
#         plt.savefig('/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+str(q)+'.png')
        
        r=1000
        rms_1000 = mc_rms (y_array1, y_array2, unc1, unc2, r)
        rms_1000s.append(rms_1000[0])
        plt.subplot(414)
        plt.hist(rms_1000[2], bins=50)
        plt.ylabel('Frequency')
        plt.xlabel('RMS')
        plt.title(str(plot_title)+str(r))
        plt.savefig('/Users/victoriaditomasso/phys767/MCSpectra/hist_'+str(plot_title)+str(r)+'.png')


#         plt.plot(shifted_w_tar[f:l], f_tar[f:l], color='black')
#         plt.plot(shifted_w_tar[f:l], f_comp_norm_dk_interp[f:l], color='red')
#         plt.show()
    df['RMS n=100'] = rms_100s
    df['RMS n=200'] = rms_200s
    df['RMS n=500'] = rms_500s
    df['RMS n=1000'] = rms_1000s
    
    df.to_csv(str(data_tar[0][1])+'_'+str(spec_order)+'testing_mc.txt',sep='\t')



Database ready for use


In [39]:
yfcq(334, 61, "/Users/victoriaditomasso/Plotting_Spectra/comp_sample_order_61_badremoved.txt")

KeyboardInterrupt: 

In [ ]:
yfcq(98, 61, "/Users/victoriaditomasso/Plotting_Spectra/comp_sample_order_61_badremoved.txt")